In [ ]:
from pyqubo import Xor, And, Or
from pyqubo import AndConst, OrConst, XorConst
from pyqubo import Binary, Spin, solve_qubo, Integer
import dimod
from pyqubo import Array

In [ ]:
from IPython.display import IFrame

## Half and Full Adder

<img src="./images/half_adder.png" alt="half-adder" align="left" style="width: 300px;"/>

In [ ]:
class HalfAdder:
    def __init__(self, a, b, s, c, label):
        xor_const = XorConst(a, b, s, label=label+'_xor')  
        and_const = AndConst(a, b, c, label=label+'_and')
        self.H = xor_const + and_const

<img src="./images/full_adder.png" alt="full-adder" align="left" style="width: 600px;"/>

In [ ]:
class FullAdder:
    def __init__(self, a, b, c_in, s, c_out, label):
        s1 = Binary(label+'_s1')
        c1 = Binary(label+'_c1')
        c2 = Binary(label+'_c2')
        h1 = HalfAdder(a, b, s1, c1, label+'_FA_HA1')
        h2 = HalfAdder(s1, c_in, s, c2, label+'_FA_HA2')
        or_const = OrConst(c1, c2, c_out, label+'_FA_or')
        self.H = h1.H + h2.H + or_const

## Two-Bit Multiplier

In [ ]:
class TwoBitMultiplier:
    
    def __init__(self, a, b, p, label):
        a0, a1 = a 
        b0, b1 = b
        p0, p1, p2, p3 = p
        
        b0a0 = Binary(label+'_b0a0') 
        b0a1 = Binary(label+'_b0a1')
        b1a0 = Binary(label+'_b1a0')
        b1a1 = p3 # Binary(f"{label}_b1a1")
        
        and_1 = AndConst(b1, a1, b1a1, label+'_and1') 
        and_2 = AndConst(b1, a0, b1a0, label+'_and2')
        and_3 = AndConst(b0, a1, b0a1, label+'_and3')
        and_4 = AndConst(a0, b0, b0a0, label+'_and4')  
        
        ha1_c = Binary("ha1_c")  
        
        ha1 = HalfAdder(b1a0, b0a1, p2, ha1_c, label+'_ha1')  # ha1_s
        ha2 = HalfAdder(b0a0, ha1_c, p1, p0, label+'_ha2')  # ha2_s
        
        self.H = and_1 + and_2 + and_3 + and_4 + ha1.H + ha2.H

In [ ]:
def find_two_bit_multiplier_solutions(num_sol=5):
    x = Array.create('x', shape=(2), vartype='BINARY')
    y = Array.create('y', shape=(2), vartype='BINARY')
    p = Array.create('p', shape=(4), vartype='BINARY')
    H = TwoBitMultiplier(x, y, p, "mult").H
    model = H.compile()
    qubo, offset = model.to_qubo()
    all_solutions = dimod.ExactSolver().sample_qubo(qubo)
    best_n_solutions = model.decode_dimod_response(all_solutions)[0:num_sol]  
    return best_n_solutions

In [ ]:
two_bit_multiplier_solutions = find_two_bit_multiplier_solutions(num_sol=5)

In [ ]:
two_bit_multiplier_solutions

## Three-Bit Multiplier

<img src="./images/multiplier.png" alt="multiplier" align="left" style="width: 700px;"/>

In [ ]:
class ThreeBitMultiplier:
    
    def __init__(self, a, b, p, label): 
        a0, a1, a2 = a 
        b0, b1, b2 = b
        p0, p1, p2, p3, p4, p5 = p
        
        b0a0 = Binary(label+"_b0a0") 
        b0a1 = Binary(label+"_b0a1")
        b1a0 = Binary(label+"_b1a0")
        b1a1 = Binary(label+"_b1a1")
        b0a2 = Binary(label+"_b0a2")
        b2a0 = Binary(label+"_b2a0")
        b1a2 = Binary(label+"_b1a2")
        b2a1 = Binary(label+"_b2a1")
        b2a2 = p5 # Binary(label+"_b2a2")
        
        and_9 = AndConst(a0, b0, b0a0, label+"_and9")  # b0a0
        and_8 = AndConst(b0, a1, b0a1, label+"_and8")
        and_7 = AndConst(b1, a0, b1a0, label+"_and7")
        and_6 = AndConst(b1, a1, b1a1, label+"_and6")
        and_5 = AndConst(b0, a2, b0a2, label+"_and5")
        and_4 = AndConst(b2, a0, b2a0, label+"_and4")
        and_3 = AndConst(b1, a2, b1a2, label+"_and3")
        and_2 = AndConst(b2, a1, b2a1, label+"_and2")
        and_1 = AndConst(b2, a2, b2a2, label+"_and1")
         
        ha1_c = Binary("ha1_c")
        fa1_c = Binary("fa1_c")
        fa1_s = Binary("fa1_s")
        ha2_c = Binary("ha2_c")
        fa2_c = Binary("fa2_c")
        fa2_s = Binary("fa2_s")
        ha3_c = Binary("ha3_c")
        
        ha1 = HalfAdder(b2a1, b1a2, p4, ha1_c, label+"_ha1")  # ha1_s
        fa1 = FullAdder(b1a1, b2a0, ha1_c, fa1_s, fa1_c, label+"_fa1")
        
        ha2 = HalfAdder(b0a2, fa1_s, p3, ha2_c, label+"_ha2")  # ha2_s
        fa2 = FullAdder(b1a0, fa1_c, ha2_c, fa2_s, fa2_c, label+"_fa2")
        
        ha3 = HalfAdder(b0a1, fa2_s, p2, ha3_c, label+"_ha3")  # ha3_s
        fa3 = FullAdder(b0a0, fa2_c, ha3_c, p1, p0, label+"_fa3")  # fa3_c, fa3_s
        
        self.H = and_1 + and_2 + and_3 + and_4 + and_5 + and_6 + and_7 + and_8 + and_9 \
        + ha1.H + ha2.H + ha3.H \
        + fa1.H + fa2.H + fa3.H

In [ ]:
def find_three_bit_multiplier_factors(product=[0, 0, 0, 0, 0, 0]):
    x = Array.create('x', shape=(3), vartype='BINARY')
    y = Array.create('y', shape=(3), vartype='BINARY')
    H = ThreeBitMultiplier(x, y, product, "mult").H
    model = H.compile()
    qubo, offset = model.to_qubo()
    sol = solve_qubo(qubo, beta_range=(100.,10000.))
    decoded_threebit = model.decode_solution(sol, vartype="BINARY")[0]
    print("x is " + "".join([str(i) for i in decoded_threebit['x'].values()]))
    print("y is " + "".join([str(i) for i in decoded_threebit['y'].values()]))

In [ ]:
find_three_bit_multiplier_factors([0, 0, 1, 0, 1, 0])